# Bioinformatics Project

## Preliminar data analysis

Our available data for the performance of prediction models consists in two dataframes. The first one, already labelled as training data, contains 38 observations of patients, to whom the expression of 7129 genes has been assessed in order to identify any characteristic expression pattern for differential diagnosis of two leucemia conditios: acute myeloid leukemia (AML) and acute lymphoid leukemia (ALL). The test data (in which the classification models are going to be tested) consist in the expression levels of the same genes for 34 different genes. The actual diagnosis of both groups is also provided.

## Data pre-processing
Due to the vast extension of the data, an initial pre-procesing is needed in order to minimize the number of features to be used with the ML tools.
Since the expression levels of many genes has been analysed, it is important to determine which of them show a correlation with the differential diagnosis of the diseases (i.e. which of them show a a significant change in the expression when the patient has been diagnosed with ALL or AML).In order to do that, it is useful t perfom a feature selection.





In [1]:
#Importing all the necessary libraries for the ML analysis
#import scipy
import numpy as np
#import matplotlib
import sklearn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
##Importar los metodos que vamos a utilizar
from sklearn.metrics import classification_report
#from pandas.plotting import scatter_matrix
#import matplotlib.pyplot as plt
#from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.neural_network import MLPClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from sklearn.naive_bayes import GaussianNB
#from sklearn.svm import SVC
import pandas as pd

C:\Users\Maria\Anaconda2\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#Reading the files:

#These paths might change according to the locations of the files within one's computer
ruta_train='C:\\Users\\Maria\\OneDrive\\Master\\Bioinformatica\\Trabajo\\Originales\\data_set_ALL_AML_train.csv'
ruta_y='C:\\Users\\Maria\\OneDrive\\Master\\Bioinformatica\\Trabajo\\Originales\\actual.csv'
ruta_test='C:\\Users\\Maria\\OneDrive\\Master\\Bioinformatica\\Trabajo\\Originales\\data_set_ALL_AML_independent.csv'
train=pd.read_csv(ruta_train)
test=pd.read_csv(ruta_test)
y=pd.read_csv(ruta_y)


The response is coded as a categorical variable, with 1 coding ALL and 0 coding AML. This response variable is split into the diagnosis of test patients and train patients

In [5]:
y.index=y['patient']
y['cancer']=np.where(y.cancer=='ALL', 1, 0) 
y.groupby('cancer').size() 
ytrain=y[:len(train)]
ytest=y[len(train):]


Features are initially placed in the rows of the dataframe, while patients are in the columns. Thus, it is necessary to transpose the data. Besides, there are some columns name 'call', between the patients information, whose data is not relevant for our model. These columns were also removed. Lastly, the name of the features was changed and the actual expression measure was converted into a numeric data.

In [3]:
#Transposing the data
train=train.T
test=test.T


#Removing call data
for fila in train.index:
    if 'call' in fila:
        train=train.drop(fila)

for fila in test.index:
    if 'call' in fila:
        test=test.drop(fila)

#Collumns are labelled with the gene accession number

columnastrain=train.loc['Gene Accession Number']
train=train[2:]
train.columns=columnastrain

columnastest=test.loc['Gene Accession Number']
test=test[2:]
test.columns=columnastest

#Converting into numeric
train=train.astype('float')
test=test.astype('float')
train.index=train.index.astype(int)
test.index=test.index.astype(int)
train=train.sort_index()
test=test.sort_index()

Once the data has the accurate structure, it is time to perform the aforementioned feature selection. Two algorithms for this aim were used in a combined way. In the first one, an univariant selection was perform, and thus each feature was individually selected or removed for the final analysis. In order to do that, the Fischer score was computed and the 100 best atributes were selected. 
On the other hand, a packing selection tool was used. This type of tools consider the selection as a searching problem (typical of the artificial intelligence tools), and different combinations of features are evaluated and compared. To each of these combinations, a score is computed and assigned and some algorithms are run to select the best combinations. In this case, the algorithm for the selection was the recursive feature removal. 

In [6]:
#Univariant feature selection (F-score)
k = 100  
columnas = list(train.columns.values)
seleccionadas = SelectKBest(f_classif, k=k).fit(train, ytrain['cancer'])
atrib = seleccionadas.get_support()
atributos1 = [columnas[i] for i in list(atrib.nonzero()[0])]

#Recursive atribute selecion
modelo = ExtraTreesClassifier()
era = RFE(modelo, 100)  # número de atributos a seleccionar
era = era.fit(train, ytrain['cancer'])
atrib2 = era.support_
atributos2 = [columnas[i] for i in list(atrib2.nonzero()[0])]

#Se combinan los tributos elegidos en una lista
print('The selected features are: ')
atribselec=list(set(atributos1)|set(atributos2))
print(atribselec)

trainred=pd.DataFrame()
for i in atribselec:
    trainred[i]=train[i]

testred=pd.DataFrame()
for i in atribselec:
    testred[i]=test[i]

The selected features are: 
['X71345_f_at', 'L38608_at', 'L41268_f_at', 'U12471_cds1_at', 'K03189_f_at', 'X58401_at', 'L08904_at', 'J03801_f_at', 'X17093_at', 'J00212_f_at', 'X67698_at', 'M33317_f_at', 'M63138_at', 'HG4535-HT4940_s_at', 'U01337_at', 'J04027_at', 'X06985_at', 'M95178_at', 'HG4490-HT4876_f_at', 'L08246_at', 'U61836_at', 'M11147_at', 'M27318_f_at', 'M21551_rna1_at', 'Z80781_at', 'L05188_f_at', 'M62762_at', 'L42583_f_at', 'M92269_f_at', 'L42611_f_at', 'Y00339_s_at', 'M32304_s_at', 'M28585_f_at', 'HG458-HT458_f_at', 'U41767_s_at', 'Z68274_at', 'V00551_f_at', 'L42379_at', 'AFFX-HUMTFRR/M11507_M_at', 'U07132_at', 'Y00787_s_at', 'X85116_rna1_s_at', 'Z80779_at', 'Z48501_s_at', 'M81933_at', 'Z49105_at', 'X05345_at', 'Z32765_at', 'X58399_at', 'X06825_at', 'M95678_at', 'D10495_at', 'U20499_at', 'M20030_f_at', 'L00389_f_at', 'X00540_at', 'D14874_at', 'HG67-HT67_f_at', 'M96956_at', 'M81695_s_at', 'X14008_rna1_f_at', 'M68891_at', 'J00148_cds2_f_at', 'U62434_at', 'X01703_at', 'U82759_

Since 100 features have been selected with each type of tool, a maximum number of 200 were considered to be significant for our classification models. They may also be too many, and so a principal component analysis (PCA) was performed. This is an statistical tool used for describing complex data in terms of new uncorrelationated variables. In this case, the algorithm is computed in a way that the new variables mantain the 95% of the variance in the original variables. Since PCA is affected by the data scale, it is necessary to standarize it previously to the computing. 

In [10]:
scaler=StandardScaler()
scaler.fit(trainred)
train=scaler.transform(trainred)
test=scaler.transform(testred)

pca=PCA(.95) 
pca.fit(trainred)

train=pca.transform(trainred)
test=pca.transform(testred)

print("TRAIN")
print(pd.DataFrame(train))
print("\n"+"-"*50+"\n")
print("TEST")
print(pd.DataFrame(test))

TRAIN
              0             1             2             3             4   \
0   -9681.082808   2534.252138  -7438.581303   3706.233034   1220.230395   
1    3160.414195   2835.256559   -552.502300  -5550.027126  -2680.097293   
2   -8927.618764   7851.116773   3743.424400  -1968.232932  -1865.951512   
3   -9747.572261    750.099330   1000.927666  -1029.336430  -1588.556038   
4   -6141.154057   1479.637519   1565.534744   1031.412861   1154.893866   
5   -7535.963940  19948.282396   6913.581629  -1211.907323   -333.304343   
6   -9826.171868   3749.250986   3008.041424    838.851110  -1459.883767   
7   -9984.403923  -1465.785398  -1727.600825  -1181.426452   1818.175027   
8   -6280.929310   5725.263461   1696.386701  -3563.730818  -3864.310488   
9   -9455.272442   3482.742231   4121.846106   -837.232867  -2277.820622   
10 -10051.249566   2923.558477   4025.725871  -3990.347136  -2384.252396   
11   1453.857547  -1723.082690   2917.544977  -1104.262623   -672.523174   
12  -9

## Machine learning algorithms to compute classifiers

Up to this point, we are now able to perform the machine learnings algorithms in order to establish a classification model for the leukemia diagnosis. 

### Decision tree
The first implemented algorithm was the decision tree classifier. which computes a diagram with logic construction in order to represent a set of conditions, which are consecutively assessed for the resolution of a problem. In order to get the best results, the parameters to be used for building the model where calculated using a cross-validation algorithm. The best parameters were used to build the final decision tree, which was evaluated with the test sample, constructing the confussion matrix. 

In [14]:
split_range=list(range(2,15))
prof_range=list(range(2, 10))

param_grid={'min_samples_split':split_range, 'max_depth':prof_range}
clf_gini = DecisionTreeClassifier(criterion = 'gini', random_state=100)
grid_dt=GridSearchCV(clf_gini, param_grid, scoring='accuracy')
grid_dt.fit(train, ytrain['cancer'])
print(grid_dt.best_score_)
print(grid_dt.best_params_)

mejor_clf_gini=grid_dt.best_estimator_


ypred=mejor_clf_gini.predict(test)
#print(mejor_clf_gini.predict_proba(test))
print("CONFUSSION MATRIX")
print("\nTEST\n")
print(pd.crosstab(ytest['cancer'],ypred, rownames=['Actual diagnosis'], colnames=['Predicted diagnosis']))
ypred_t=mejor_clf_gini.predict(train)
print("\nTRAIN\n")
print(pd.crosstab(ytrain['cancer'],ypred_t, rownames=['Actual diagnosis'], colnames=['Predicted diagnosis']))

print("\nCLASSIFICATION REPORT\n")
print(classification_report(ytest['cancer'], ypred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(ytest['cancer'], ypred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("\nAUROC\n")
print(roc_auc)

0.9473684210526315
{'min_samples_split': 2, 'max_depth': 2}
CONFUSSION MATRIX

TEST

Predicted diagnosis   0   1
Actual diagnosis           
0                    11   3
1                     0  20

TRAIN

Predicted diagnosis   0   1
Actual diagnosis           
0                    11   0
1                     0  27

CLASSIFICATION REPORT

             precision    recall  f1-score   support

          0       1.00      0.79      0.88        14
          1       0.87      1.00      0.93        20

avg / total       0.92      0.91      0.91        34


AUROC

0.8928571428571428


The calculated decision tree is not a bad classifier, but there is some kind of overfitting. When we apply this model to the training set, the accuracy of it is perfect: every patient is correctly diagnose. However, when running the classifier model for the test set, 3 patients are misdiagnosed.

### Random forest
Instead of using a single decision tree, this algorithm computes a whole forest of trees with little depth. In order to obtain the classification result, it takes the individual result of each tree and the resulting class is the most "voted" one. 
In order to tackle the aforementioned overfitting, a cross-validation search of the best hypeparameters was also performed. 
Due to the high number of hyperparameters to be tested, two tipes of parameter search are computed. In first place, a random search is perfomr in order to get close to the actual best value of each parameter. Once we have some idea of this approximate value, we perform the grid search by building a grid within a range including this approximation. Best computed parameters are finally used to compute the final ranfom forest classifier.

In [16]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_clf=RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf_clf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=100, n_jobs = -1)
rf_random.fit(train, ytrain['cancer'])



print(rf_random.best_params_)

param_grid = {
    'bootstrap': [True],
    'max_depth': [None],
    'max_features': ['auto'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 4, 6],
    'n_estimators': [500, 1000, 2000]
}

grid_rf=GridSearchCV(rf_clf, param_grid, scoring='accuracy', cv=3)
grid_rf.fit(train, ytrain['cancer'])
print(grid_rf.best_score_)
print(grid_rf.best_params_)

mejor_rf_clf=grid_rf.best_estimator_


ypred_rf=mejor_rf_clf.predict(test)

print("CONFUSSION MATRIX")
print("\nTEST\n")
print(pd.crosstab(ytest['cancer'],ypred_rf, rownames=['Actual diagnosis'], colnames=['Predicted diagnosis']))
ypred_t_rf=mejor_rf_clf.predict(train)
print("\nTRAIN\n")
print(pd.crosstab(ytrain['cancer'],ypred_t_rf, rownames=['Actual diagnosis'], colnames=['Predicted diagnosis']))

print("\nCLASSIFICATION REPORT\n")
print(classification_report(ytest['cancer'], ypred_rf))
false_positive_rate, true_positive_rate, thresholds = roc_curve(ytest['cancer'], ypred_rf)
roc_auc_rf = auc(false_positive_rate, true_positive_rate)
print("\nAUROC\n")
print(roc_auc_rf)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.0min finished


{'bootstrap': False, 'min_samples_leaf': 2, 'n_estimators': 1600, 'max_features': 'sqrt', 'min_samples_split': 10, 'max_depth': 100}
0.9473684210526315
{'bootstrap': True, 'min_samples_leaf': 1, 'n_estimators': 1000, 'min_samples_split': 2, 'max_features': 'auto', 'max_depth': None}
CONFUSSION MATRIX

TEST

Predicted diagnosis   0   1
Actual diagnosis           
0                    12   2
1                     0  20

TRAIN

Predicted diagnosis   0   1
Actual diagnosis           
0                    11   0
1                     0  27

CLASSIFICATION REPORT

             precision    recall  f1-score   support

          0       1.00      0.86      0.92        14
          1       0.91      1.00      0.95        20

avg / total       0.95      0.94      0.94        34


AUROC

0.9285714285714286


### Neural network 
This calssification model is based in the biological neurons. It recieves an input and produces a signal (output) based on it wich is recived by another neuron as a new input. Each neuron has an activation function, which determines whether an output is computed for a determined input will be sended to another neuron or not. Due to the complexity of this algorithm, in this case no parameter search was computed. 

In [17]:
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
mlp.fit(train, ytrain['cancer'])
ypred_nn=mlp.predict(test)

print("CONFUSSION MATRIX")
print("\nTEST\n")
print(pd.crosstab(ytest['cancer'],ypred_nn, rownames=['Actual diagnosis'], colnames=['Predicted diagnosis']))
ypred_t_nn=mlp.predict(train)
print("\nTRAIN\n")
print(pd.crosstab(ytrain['cancer'],ypred_t_nn, rownames=['Actual diagnosis'], colnames=['Predicted diagnosis']))

print("\nCLASSIFICATION REPORT\n")
print(classification_report(ytest['cancer'], ypred_nn))
false_positive_rate, true_positive_rate, thresholds = roc_curve(ytest['cancer'], ypred_rf)
roc_auc_nn = auc(false_positive_rate, true_positive_rate)
print("\nAUROC\n")
print(roc_auc_nn)


CONFUSSION MATRIX

TEST

Predicted diagnosis  0   1
Actual diagnosis          
0                    6   8
1                    2  18

TRAIN

Predicted diagnosis  0   1
Actual diagnosis          
0                    2   9
1                    2  25

CLASSIFICATION REPORT

             precision    recall  f1-score   support

          0       0.75      0.43      0.55        14
          1       0.69      0.90      0.78        20

avg / total       0.72      0.71      0.68        34


AUROC

0.9285714285714286
